In [10]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, Subset
from PIL import Image
import torch
import os

def custom_gray_loader(path: str):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('L')  # Grayscale

def get_dataloaders(data_dir, batch_size=32):
    # === Step 1: Temporary transform for stats ===
    temp_transform = transforms.Compose([
        transforms.Resize((180, 180)),
        transforms.ToTensor()
    ])
    
    dataset_temp = datasets.ImageFolder(root=data_dir, transform=temp_transform, loader=custom_gray_loader)
    
    # === Step 2: Split the dataset ===
    total_length = len(dataset_temp)
    train_len = int(0.7 * total_length)
    val_len = int(0.2 * total_length)
    test_len = total_length - train_len - val_len

    train_set, val_set, test_set = random_split(dataset_temp, [train_len, val_len, test_len],
                                                generator=torch.Generator().manual_seed(42))

    # === Step 3: Compute mean & std on training set ===
    loader_for_stats = DataLoader(train_set, batch_size=batch_size)
    mean, std, total_pixels = 0.0, 0.0, 0

    for images, _ in loader_for_stats:
        mean += images.sum().item()
        std += (images ** 2).sum().item()
        total_pixels += images.numel()

    mean /= total_pixels
    std = (std / total_pixels - mean ** 2) ** 0.5

    print(f"📊 Mean: {mean:.4f}, Std: {std:.4f}")

    # === Step 4: Final transforms — gentle, FCN-safe, CNN-compatible ===
    transform_train = transforms.Compose([
        transforms.Resize((180, 180)),
        transforms.RandomRotation(degrees=10),                      # Safe small rotation
        transforms.RandomApply([
            transforms.RandomAffine(degrees=0, translate=(0.05, 0), fill=0)
        ], p=0.1),                                                  # Tiny shifts
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean], std=[std])
    ])

    transform_val_test = transforms.Compose([
        transforms.Resize((180, 180)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean], std=[std])
    ])

    # === Step 5: Reload dataset with new transforms ===
    full_dataset = datasets.ImageFolder(root=data_dir, loader=custom_gray_loader)

    full_dataset.transform = transform_train
    train_set.dataset = full_dataset

    full_dataset.transform = transform_val_test
    val_set.dataset = full_dataset
    test_set.dataset = full_dataset

    # === Step 6: Create dataloaders ===
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, mean, std


# Net1


In [11]:
import torch.nn as nn
import torch.nn.functional as F

class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(180 * 180, 512)      # Hidden layer 1
        self.fc2 = nn.Linear(512, 128)            # Hidden layer 2
        self.out = nn.Linear(128, 10)             # Output layer
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)     # Flatten input
        x = self.relu(self.fc1(x))   # Hidden layer 1 + activation
        x = self.relu(self.fc2(x))   # Hidden layer 2 + activation
        return self.out(x)           # Output logits

In [14]:
def evaluate_accuracy(model, data_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

def train_model(model, train_loader, val_loader, optimizer, loss_fn, device, epochs=50):
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        val_acc = evaluate_accuracy(model, val_loader, device)
        print(f"📅 Epoch {epoch+1}/{epochs} | Loss: {total_loss:.4f} | Val Acc: {val_acc:.2%}")


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_path = "./Data/images_original"
train_loader, val_loader, test_loader, mean, std = get_dataloaders(data_path, batch_size=32)

model = Net1().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # 🔥 Lower LR!
loss_fn = nn.CrossEntropyLoss()

train_model(model, train_loader, val_loader, optimizer, loss_fn, device, epochs=50)

# I want to save the model after training
torch.save(model.state_dict(), 'Net1-50epoch.pth')

print(f"✅ Train Acc: {evaluate_accuracy(model, train_loader, device):.2%}")
print(f"✅ Val Acc: {evaluate_accuracy(model, val_loader, device):.2%}")
print(f"✅ Test Acc: {evaluate_accuracy(model, test_loader, device):.2%}")


📊 Mean: 0.4673, Std: 0.4524
📅 Epoch 1/80 | Loss: 52.7691 | Val Acc: 18.09%
📅 Epoch 2/80 | Loss: 49.3363 | Val Acc: 18.09%
📅 Epoch 3/80 | Loss: 46.9974 | Val Acc: 16.58%
📅 Epoch 4/80 | Loss: 45.8357 | Val Acc: 22.11%
📅 Epoch 5/80 | Loss: 44.3001 | Val Acc: 23.62%
📅 Epoch 6/80 | Loss: 43.3984 | Val Acc: 28.64%
📅 Epoch 7/80 | Loss: 43.0898 | Val Acc: 26.63%
📅 Epoch 8/80 | Loss: 41.5627 | Val Acc: 23.12%
📅 Epoch 9/80 | Loss: 41.4709 | Val Acc: 30.65%
📅 Epoch 10/80 | Loss: 40.3104 | Val Acc: 27.14%
📅 Epoch 11/80 | Loss: 40.7025 | Val Acc: 28.64%
📅 Epoch 12/80 | Loss: 40.0097 | Val Acc: 27.14%
📅 Epoch 13/80 | Loss: 38.8195 | Val Acc: 26.13%
📅 Epoch 14/80 | Loss: 38.5824 | Val Acc: 29.65%
📅 Epoch 15/80 | Loss: 38.6603 | Val Acc: 33.17%
📅 Epoch 16/80 | Loss: 37.7951 | Val Acc: 34.17%
📅 Epoch 17/80 | Loss: 37.3837 | Val Acc: 33.17%
📅 Epoch 18/80 | Loss: 37.5153 | Val Acc: 32.16%
📅 Epoch 19/80 | Loss: 37.2706 | Val Acc: 26.13%
📅 Epoch 20/80 | Loss: 36.9550 | Val Acc: 30.15%
📅 Epoch 21/80 | Loss:

KeyboardInterrupt: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_path = "./Data/images_original"
train_loader, val_loader, test_loader, mean, std = get_dataloaders(data_path, batch_size=32)

model = Net1().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # 🔥 Lower LR!
loss_fn = nn.CrossEntropyLoss()

train_model(model, train_loader, val_loader, optimizer, loss_fn, device, epochs=100)

# I want to save the model after training
torch.save(model.state_dict(), 'Net1-100epoch.pth')

print(f"✅ Train Acc: {evaluate_accuracy(model, train_loader, device):.2%}")
print(f"✅ Val Acc: {evaluate_accuracy(model, val_loader, device):.2%}")
print(f"✅ Test Acc: {evaluate_accuracy(model, test_loader, device):.2%}")
